In [ ]:
%%bash
mkdir data
cd data
wget http://biodatageeks.ii.pw.edu.pl/sequila/data/NA12878.multichrom.md.bam
wget http://biodatageeks.ii.pw.edu.pl/sequila/data/NA12878.multichrom.md.bam.bai
wget http://biodatageeks.ii.pw.edu.pl/sequila/data/Homo_sapiens_assembly18_chr1_chrM.small.fasta
wget http://biodatageeks.ii.pw.edu.pl/sequila/data/Homo_sapiens_assembly18_chr1_chrM.small.fasta.fai

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.7.tgz
!tar xf spark-2.4.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
!pip install pysequila

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.3-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS']='--packages org.biodatageeks:sequila_2.11:0.5.16 -v pyspark-shell'

In [ ]:
import os

base_path = f"{os.getcwd()}/data"

bam_path = f'{base_path}/NA12878.multichrom.md.bam'
ref_path=f'{base_path}/Homo_sapiens_assembly18_chr1_chrM.small.fasta'
sample_id= 'NA12878'
table_name = 'reads'

app_name = "sequila"


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.appName(f'{app_name}') \
.getOrCreate()

In [ ]:
from pysequila import SequilaSession
ss = SequilaSession(spark)

ss.sql(f'''CREATE TABLE IF NOT EXISTS {table_name} \
         USING org.biodatageeks.sequila.datasources.BAM.BAMDataSource \
         OPTIONS(path "{bam_path}")''')

In [ ]:
import pandas as pd
pd.options.display.max_columns = None
ss.sql(f"SELECT * FROM {table_name} LIMIT 5").toPandas()

In [ ]:
ss.sql(f"SELECT * FROM  pileup('{table_name}', '{sample_id}', '{ref_path}', true) LIMIT 10").toPandas()


In [ ]:
ss.stop()